In [ ]:
import numpy as np
from numpy import linalg as la

np.set_printoptions(linewidth=200)

def AdjointMatrix(A):
    return np.conjugate(A.T)

In [ ]:
def Householder(A):
    """Computes a QR-decomposition of the given matrix using the Householder algorithm.
       Returns (normalList, R), where normalList is a list of Householder normal vectors
       and R is an upper triangular matrix shaped like A.
       \sa ComputeQ
    """
    A = np.array(A, copy=True)
    m, n = A.shape
    R = A.copy()
    normalList = []

    for k in range(min(m, n)):
        # Betrachte die k-te Spalte ab Zeile k
        x = R[k:, k]
        norm_x = la.norm(x)
        if norm_x == 0:
            # nichts zu tun, Spalte ist bereits null
            continue

        # Komplexes Vorzeichen (Phase) von x[0]
        if x[0] == 0:
            s = 1.0
        else:
            s = x[0] / abs(x[0])

        # Zielwert für den ersten Eintrag nach der Spiegelung
        alpha = -s * norm_x

        # Householder-Normalenvektor im verkürzten Block
        v_sub = x.copy()
        v_sub[0] -= alpha

        if la.norm(v_sub) == 0:
            continue

        # Auf volle Länge m auffüllen
        v = np.zeros(m, dtype=R.dtype)
        v[k:] = v_sub
        normalList.append(v)

        # Householder-Spiegelung von links auf R anwenden:
        # R <- (I - 2 v v* / (v* v)) R
        denom = np.vdot(v, v)  # v* v (Skalar, komplex möglich)
        if denom == 0:
            continue

        R = R - (2.0 / denom) * np.outer(v, v.conj() @ R)

    return normalList, R


def ComputeQ(normalList):
    """Given a normal list such as the one returned by Householder this function
       computes the corresponding unitary matrix Q (m x m) with orthonormal columns."""
    if not normalList:
        return np.eye(0)

    m = normalList[0].shape[0]
    dtype = normalList[0].dtype

    # S wird Q* (Produkt der Householder-Matrizen in der gleichen Reihenfolge wie bei R)
    S = np.eye(m, dtype=dtype)

    for v in normalList:
        if np.allclose(v, 0):
            continue
        denom = np.vdot(v, v)
        if denom == 0:
            continue
        S = S - (2.0 / denom) * np.outer(v, v.conj() @ S)

    # Q = S*  (Adjungierte)
    Q = np.conjugate(S.T)
    return Q


In [ ]:
def format_float(value):
    return f"{value:.4e}"

if __name__=="__main__":
    np.set_printoptions(precision=4)
    A = np.random.rand(4,3) + 1.0j * np.random.rand(4,3)
    normalList, R = Householder(A)
    Q = ComputeQ(normalList)
    print("The R should be upper triangular:")
    print(np.round(R, 10))
    if np.allclose(np.tril(R, -1), 0):
        print("✔️ R is upper triangular")
    else:
        print("❌ R is not upper triangular")
    print("If Q is unitary, the following is near zero:")
    print(format_float(la.norm(AdjointMatrix(Q) @ Q - np.eye(Q.shape[0]))))
    if np.allclose(la.norm(AdjointMatrix(Q) @ Q - np.eye(Q.shape[0])), 0):
        print("✔️ Q.T Q = I")
    else:
        print("❌ Q.T Q != I")
    print("If the solution consitutes a decomposition, the following is near zero:")
    print(format_float(la.norm(A - Q @ R)))
    if np.allclose(la.norm(A - Q @ R), 0):
        print("✔️ QR = A")
    else:
        print("❌ QR != A")